<a href="https://colab.research.google.com/github/tyoc213/fastai_xla_extensions/blob/fix_prev_lenet/explore_nbs/PickableOpt%2BBasic_lenet_exploration_MultiTPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install fastai2 from github

In [1]:
!pip install -U pandas --upgrade
!pip install -U fastcore --upgrade
!pip install -U fastai --upgrade 
!pip install -Uqq git+https://github.com/tyoc213/fastai_xla_extensions@fix_prev_lenet

Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.2)
Requirement already up-to-date: fastcore in /usr/local/lib/python3.6/dist-packages (1.0.9)
Requirement already up-to-date: fastai in /usr/local/lib/python3.6/dist-packages (2.0.10)


In [2]:
VERSION = "20200707"  #"20200515" @param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  26921      0 --:--:-- --:--:-- --:--:-- 26921


In [3]:
#!TORCH_SHOW_CPP_STACKTRACES=1 python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!python pytorch-xla-env-setup.py  --version $VERSION --apt-packages libomp5 libopenblas-dev

Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200707 ...
Uninstalling torch-1.7.0a0+12b5bdc:
  Successfully uninstalled torch-1.7.0a0+12b5bdc
Uninstalling torchvision-0.8.0a0+86b6c3e:
  Successfully uninstalled torchvision-0.8.0a0+86b6c3e
Copying gs://tpu-pytorch/wheels/torch-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
- [1 files][107.5 MiB/107.5 MiB]                                                
Operation completed over 1 objects/107.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
\ [1 files][123.8 MiB/123.8 MiB]                                                
Operation completed over 1 objects/123.8 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
/ [1 files][  2.2 MiB/  2.2 MiB]                                                
Operation completed over 1 objects/2.2 MiB.        

In [4]:
!pip freeze | grep torch 
!pip freeze | grep fast

torch==1.7.0a0+12b5bdc
torch-xla==1.6+5430aca
torchsummary==1.5.1
torchtext==0.3.1
torchvision==0.8.0a0+86b6c3e
fastai==2.0.10
fastai-xla-extensions==0.0.1
fastcore==1.0.9
fastdtw==0.3.4
fastprogress==1.0.0
fastrlock==0.5


In [5]:
from fastai.optimizer import Optimizer

In [6]:
import fastai_xla_extensions.core

In [7]:
from fastai.vision.all import *

In [8]:
default_device()

device(type='cpu')

In [9]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path; path.ls()

(#3) [Path('labels.csv'),Path('valid'),Path('train')]

In [10]:
(path/'train').ls()

(#2) [Path('train/3'),Path('train/7')]

# multi TPU

In [11]:
# Configures training (and evaluation) parameters
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch_xla.distributed.parallel_loader as pl
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
from fastai.vision.all import *
import time
from fastai.test_utils import *
print(f'torch version {torch.__version__}')

import pdb

path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path; path.ls()

def debug_on(*exceptions):
    if not exceptions:
        exceptions = (AssertionError, )
    def decorator(f):
        @functools.wraps(f)
        def wrapper(*args, **kwargs):
            try:
                return f(*args, **kwargs)
            except exceptions:
                pdb.post_mortem(sys.exc_info()[2])
        return wrapper
    return decorator


class Lenet2(nn.Module):
    def __init__(self):
        super(Lenet2, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2) # Only 2 outputs instead of 10
    @debug_on(KeyError)
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    @debug_on(KeyError)
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features





def map_fn(index, flags):
  # from fastai.callback.all import *
  dede = xm.xla_device()
  print(f'index is {index} and flags are {flags}')
  #xm.rendezvous('init')

  if not xm.is_master_ordinal():
    print(f"this is {dede}:{index} entering download once")
    xm.rendezvous('download_only_once')
    
  dblock = DataBlock(
    splitter = GrandparentSplitter(),
    item_tfms = Resize(28),
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    get_y = parent_label,
    batch_tfms = []
  )
  if xm.is_master_ordinal():
    xm.master_print(f'this is {dede} exiting download once')
    xm.rendezvous('download_only_once')
  xm.master_print('creating lenet_tpu')
  lenet_tpu = Lenet2()
  xm.master_print('lenet created, goiing for dls_tpu')
  dls_tpu = dblock.dataloaders(path, device=dede)
  xm.master_print(f'creating learner!!! for {dede}')
  

  tpu_learner = Learner(dls_tpu,
                      lenet_tpu,
                      metrics=accuracy, 
                      loss_func=F.cross_entropy,
                      cbs=[])
  print(f"################ fit for {dede}")
  xm.master_print(f'***** fit for {dede}')
  tpu_learner.fit(1, cbs=[fastai_xla_extensions.core.XLAOptCallback()])
  xm.master_print(f'***** end fit for {dede}')
  t = torch.randn((2, 2), device=dede)
  print("################Process", index ,"is using", xm.xla_real_devices([str(dede)])[0])






# https://stackoverflow.com/a/9929970/682603
# excepthook
# 
import traceback
import logging
import os, sys

def my_excepthook(excType, excValue, traceback, logger):
    print("=== *** @@@ ### %%% === *** @@@ ### %%% === *** @@@ ### %%% === *** @@@ ### %%% === *** @@@ ### %%% === *** @@@ ### %%% Logging an uncaught exception",
                 exc_info=(excType, excValue, traceback))

sys.excepthook = my_excepthook
sys.unraisablehook = my_excepthook
##############threading.excepthook
#https://docs.python.org/3/library/sys.html#sys.excepthook




print('launching n procs')

flags={}
flags['batch_size'] = 32
flags['num_workers'] = 8
flags['num_epochs'] = 1
flags['seed'] = 1234

xmp.spawn(map_fn, args=(flags,), nprocs=8, start_method='fork')
print('end of launch')

torch version 1.7.0a0+12b5bdc
launching n procs
index is 0 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:1 exiting download once
index is 6 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:6 entering download once
index is 4 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:4 entering download once
index is 7 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:7 entering download once
index is 2 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:2 entering download once
index is 1 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:1 entering download once
index is 3 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:3 entering download once
index is 5 and flags are {'batc

epoch,train_loss,valid_loss,accuracy,time
0,0.697004,0.696380,0.495584,02:54


epoch,train_loss,valid_loss,accuracy,time
0,0.696158,0.696380,0.495584,02:54


################ fit for xla:0
################ fit for xla:0
################ fit for xla:0


epoch,train_loss,valid_loss,accuracy,time
0,0.696032,0.696380,0.495584,02:54


...

epoch,train_loss,valid_loss,accuracy,time
0,0.696967,0.696380,0.495584,02:54


################ fit for xla:0
################ fit for xla:0


epoch,train_loss,valid_loss,accuracy,time
0,0.696473,0.696380,0.495584,02:54


epoch,train_loss,valid_loss,accuracy,time
0,0.696943,0.696380,0.495584,02:54


***** end fit for xla:1
################Process 7 is using TPU:7
################Process 4 is using TPU:4
################Process 5 is using TPU:5
################Process 0 is using TPU:0
################Process 6 is using TPU:6
################Process 3 is using TPU:3
################Process 2 is using TPU:2
################Process 1 is using TPU:1
end of launch
